# Colon glands segmentation with U-Net
<img src="images/glas_header.bmp" style="width:100%;" height="250" align="center">

In this assignment we are going to develop **U-Net** models to segment benign and malignant glands in histopathology images of colon tissue, stained with hematoxylin and eosin (H&E).

## Teaching assistants

- Hans Pinckaers: Hans.Pinckaers@radboudumc.nl 
- Thomas De Bel: Thomas.deBel@radboudumc.nl 

## Students
Please fill in this cell with your name and e-mail address. This information will be used to grade your assignment.

* Name student #1, email address: ...
* Name student #2, email address: ...
* Name student #3, email address: ...

Please submit your notebook via grand-challenge.org: https://ismi-glas.grand-challenge.org/

This notebook requires tensorflow and keras installed, please make sure you installed the required packages: https://ismi-nodules.grand-challenge.org/Resources/

Submit a notebook **WITH ALL CELLS EXECUTED!!!**

* Groups: You should work in pairs or alone. Working in groups of 2-3 is preferable.
* Deadline for this assignment: 
 * Friday, March 22nd until 23:59h.
 * 5 points (maximum grade = 100 points) penalization per day after deadline.
* Submit your **fully executed** notebook to the grand-challenge.org platform ** AND MAIL THE SOLUTION TO THE STUDENT ASSISTANTS**
* The file name of the notebook you submit **must be** ```NameSurname1_NameSurname2_NameSurname3.ipynb```

## Data
<img src="images/instance_segmentation.png" width="500" height="250" align="right">
For this assignment we will use data from the publicly available GlaS dataset (https://warwick.ac.uk/fac/sci/dcs/research/tia/glascontest/).
The original GlaS dataset consists of 165 images, 85 used for training and 80 used for testing. Each case contains:
* a small crop of histopathology (RGB) image
* manual annotations of colon glands, both benign and malignant

Similar to what done in the original U-Net paper, the main task of the GlaS challenge was **instance segmentation**, meaning that participants in the challenge had to build a model that was good at separating glands from background (segmentation) but also at identifying/detecting glands. Therefore, the type of manual annotations that come with the original GlaS dataset is similar to what shown in the image on the right in this cell. You can clearly see that each gland in the manually annotated mask gets a different label. Additionally, the size of images in the original dataset is not fixed and is approximately 500 x 700 pixels.

### "Binary GlaS 256"
In this assignment, we have simplified things a bit. In particular:
* we have resized all images to 256x256, both in the training and in the test set
* we have converted all manual annotations to a binary form, removing the task of instance segmentation

Examples of images in the dataset of this assignment (which we can call "binary glas 256") are the following:

<table>
<tr>
<td><img src="images/1.png" width="120" height="120" align="left"></td>
<td><img src="images/1_anno.png" width="120" height="120" align="left"></td>
<td><img src="images/2.png" width="120" height="120" align="left"></td>
<td><img src="images/2_anno.png" width="120" height="120" align="left"></td>
<td><img src="images/3.png" width="120" height="120" align="left"></td>
<td><img src="images/3_anno.png" width="120" height="120" align="left"></td>
<td><img src="images/4.png" width="120" height="120" align="left"></td>
<td><img src="images/4_anno.png" width="120" height="120" align="left"></td>
</tr>
</table>


This means that the datasets you will be downloading in this notebook are a modified version of the original ones, and are only valid for the purpose of this assignment. Also, the results you will be getting in this notebook are not directly applicable to the leaderboard of the official GlaS challenge. Still, the setting we propose is a good playground to experiment with the U-Net model, which is the main goal of this assignment.


# Tasks
The main tasks of this assignment are the following:

### 1. Train a baseline U-Net model (5 points)
### 2. Implement a basic ``unet_block`` with batch normalization, build and train a new U-Net model (30 points)
### 3. Add regularization by adding dropout to the unet_block and an L2 regularization loss (25 points)
### 4. Use weight maps (25 points)
### 5. Use valid convolutions (35 points)

Let's get started!

## Load libraries and data

In [ ]:
# Import all libraries
import requests
from tqdm import trange
import zipfile
import os
from PIL import Image
import numpy as np
import pylab as plt
import matplotlib
from random import randint
from scipy.ndimage.interpolation import rotate
from skimage.transform import rescale, resize
from IPython.display import clear_output
import random
import shutil
%matplotlib inline

import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Cropping2D, Reshape, BatchNormalization
from keras import optimizers
from keras.optimizers import SGD, Adam
import keras.callbacks
from keras import backend as K


# this part is needed if you run the notebook on Cartesius with multiple cores
n_cores = 32
config = tf.ConfigProto(intra_op_parallelism_threads=n_cores-1, inter_op_parallelism_threads=1, allow_soft_placement=True)
session = tf.Session(config=config)
K.set_session(session)
os.environ["OMP_NUM_THREADS"] = str(n_cores-1)
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

In [ ]:
# define your local directory where data is stored
data_dir = './'

In [ ]:
# Download GlaS dataset resized to 256x256 from SURFDrive
link = 'https://surfdrive.surf.nl/files/index.php/s/D5TLR0rPaUogqr7/download'
file_name = "glas_256.zip"
with open(file_name, "wb") as f:
        response = requests.get(link, stream=True)
        total_length = response.headers.get('content-length')
        if total_length is None: # no content length header
            f.write(response.content)
        else:
            dl = 0
            total_length = int(total_length)
            for data in response.iter_content(chunk_size=4096):
                dl += len(data)
                f.write(data)
with zipfile.ZipFile(file_name,"r") as zip_ref:
    zip_ref.extractall(data_dir)
os.remove(os.path.join(data_dir, file_name))

Let's define some classes and functions that will be used throughout this assignment. Some classes, like the ``Dataset`` class, have been used already in previous assignments.

In [ ]:
class DataSet:
    
    def __init__(self, imgs, lbls=None):
        self.imgs = imgs
        self.lbls = lbls
    
    def get_lenght(self):
        return len(self.imgs)
    
    def show_image(self, i):
        if self.lbls != None:
            f, axes = plt.subplots(1, 2)
            for ax, im, t in zip(axes, 
                                 (self.imgs[i], self.lbls[i]), 
                                 ('RGB image', 
                                  'Manual annotation; Range: [{}, {}]'.format(self.lbls[i].min(), 
                                                                              self.lbls[i].max()))):
                ax.imshow(im)
                ax.set_title(t)
        else:
            plt.imshow(self.imgs[i])
            plt.title('RGB image')
        plt.show()

In [ ]:
def load_img(path):
    return np.array(Image.open(path))

For training purposes, we have to extract patches from the training set to build our model. Therefore, we have to define a patch extractor and a batch generator. Luckily, we can reuse some of the code used in previous assignments.

In [ ]:
class PatchExtractor:

    def __init__(self, patch_size, flipping=True):
        self.patch_size = patch_size 
        self.flipping = flipping
    
    def get_patch(self, image, label):
        ''' 
        Get a patch of patch_size from input image, along with corresponding label map.
        This function works with image size >= patch_size, and pick random location of the patch inside the image.
        (Possibly) return a flipped version of the image and corresponding label.

        image: a numpy array representing the input image
        label: a numpy array representing the labels corresponding to input image
        '''
        
        # pick a random location
        dims = image.shape        
        r = randint(0, dims[0]-self.patch_size[0])
        c = randint(0, dims[1]-self.patch_size[1])
        
        patch = image[r:r+self.patch_size[0], c:c+self.patch_size[1], :]
        target = label[r:r+self.patch_size[0], c:c+self.patch_size[1]].reshape(self.patch_size[0], self.patch_size[1], 1)

        patch_out = patch / 255. # normalize image intensity to range [0., 1.]
        target_out = target
        
        # random flipping
        flip1 = self.flipping and random.random() > 0.5
        flip2 = self.flipping and random.random() > 0.5
        if flip1:
            patch_out = np.fliplr(patch_out)
            target_out = np.fliplr(target_out)
        if flip2:
            patch_out = np.flipud(patch_out)
            target_out = np.flipud(target_out)
            
        return patch_out, target_out

In [ ]:
class BatchCreator:
    
    def __init__(self, patch_extractor, dataset, target_size):
        self.patch_extractor = patch_extractor
        self.target_size = target_size # size of the output, can be useful when valid convolutions are used
        
        self.imgs = dataset.imgs
        self.lbls = dataset.lbls
                
        self.n = len(self.imgs)
        self.patch_size = self.patch_extractor.patch_size
    
    def create_image_batch(self, batch_size):
        '''
        returns a single augmented image (x) with corresponding labels (y) in one-hot structure
        '''
        
        x_data = np.zeros((batch_size, *self.patch_extractor.patch_size, 3))
        y_data = np.zeros((batch_size, *self.target_size, 2)) # one-hot encoding with 2 classes
        
        for i in range(0, batch_size):
        
            random_index = np.random.choice(len(self.imgs)) # pick random image
            img, lbl = self.imgs[random_index], self.lbls[random_index] # get image and segmentation map
            patch_img, patch_lbl = self.patch_extractor.get_patch(img, lbl) # when image size is equal to patch size, this line is useless...
        
            # crop labels based on target_size
            h, w, _ = patch_lbl.shape
            ph = (self.patch_extractor.patch_size[0] - self.target_size[0]) // 2
            pw = (self.patch_extractor.patch_size[1] - self.target_size[1]) // 2
            x_data[i, :, :, :] = patch_img
            y_data[i, :, :, 0] = 1 - patch_lbl[ph:ph+self.target_size[0], pw:pw+self.target_size[1]].squeeze()
            y_data[i, :, :, 1] = patch_lbl[ph:ph+self.target_size[0], pw:pw+self.target_size[1]].squeeze()
        
        return (x_data.astype(np.float32), y_data.astype(np.float32))
    
    def get_image_generator(self, batch_size):
        '''returns a generator that will yield image-batches infinitely'''
        while True:
            yield self.create_image_batch(batch_size)

Since the output of U-Net may not have the same size as its input, let's define a function ``pad_prediction()`` that visualizes labels adding padding, to match the input size.

In [ ]:
def pad_prediction(prediction, input_size, pad_with=-1.0):
    """Only for visualization purpose, it introduces artificial -1."""
    pad_pred = pad_with * np.ones(input_size).astype(float)
    pred_size = prediction.shape
    D = ((input_size[0]-pred_size[0])//2, (input_size[1]-pred_size[1])//2)
    pad_pred[D[0]:D[0]+pred_size[0], D[1]:D[1]+pred_size[1]] = prediction
    return pad_pred

As we have seen, manual annotations in the GlaS challenge come with a different label per gland instance. This is because the main task of the original GlaS challenge was instance segmentation, meaning segmentation of glands as well as detection of each gland as a separate object. We are not going to focus on the instance segmentation part for the time being, therefore we can just treat all manually annotated objects as belonging to the same class. Therefore, we have modified the labels in the dataset by just using label=1 for glands and label=0 for background. When we saved this map to disk, background got value 0 and foreground got value 255 (as you can see in the previous cells showing manual annotations of trainign and validation images). In order to be able ot use these annotations, we have to convert them to [0, 1] values. For this purpose, we define the following function, wich we will apply to the validation and to the training set.

In [ ]:
def binarize_dataset(dataset): 
    return DataSet(dataset.imgs, [(x>0).astype(int) for x in dataset.lbls])

Let's also we define a function that allows to apply the model to a given dataset, which will be used a lot to test your models and visualize results. Note that this function includes a parameter to make a submission file, which will be called ``results.zip``, to be submitted to grand-challenge. By default, this parameter is set to ``False``. Enable it later to make subsmission files.

In [ ]:
def apply_model(model, dataset, experiment_name='basic_unet', make_submission_file=False):
    """Apply a given model to the test set, optionally makes a submission file in ZIP format."""
    
    output_dir = os.path.join(data_dir, experiment_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in range(len(dataset.imgs)):
        fig = plt.figure(figsize=(10,20))
        input_img = np.expand_dims(dataset.imgs[i], axis=0)/255.
        output = model.predict(input_img, batch_size=1)[0, :, :, :]
        plt.subplot(1, 2, 1); plt.imshow(dataset.imgs[i])
        plt.subplot(1, 2, 2); plt.imshow(np.argmax(output, axis=2))
        if make_submission_file:
            prediction = Image.fromarray(np.argmax(output, axis=2).astype(np.uint8))
            prediction.save(os.path.join(output_dir, '{}.png'.format(i)))
        plt.show()
        
    if make_submission_file:
        shutil.make_archive('results', 'zip', output_dir)

Let's also define a Logger class that will store useful information during training:

In [ ]:
def crop(masks, lost_border):
    ph = lost_border[0] // 2
    pw = lost_border[1] // 2
    h, w = masks[0].shape    
    return np.array(masks)[:, ph:h-ph, pw:w-pw]   

def calculate_dice(x, y):
    '''returns the dice similarity score, between two boolean arrays'''
    return 2 * np.count_nonzero(x & y) / (np.count_nonzero(x) + np.count_nonzero(y))
    
class Logger(keras.callbacks.Callback):

    def __init__(self, validation_data, lost_border, data_dir, model_name):
        self.val_imgs = np.array(validation_data.imgs) / 255.
        self.val_lbls = crop(validation_data.lbls, lost_border)
        self.model_filename = os.path.join(data_dir, model_name + '.h5')
        
        self.losses = []
        self.dices = []
        self.best_dice = 0
        self.best_model = None
        
        self.predictions = None
    
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.losses.append(logs.get('acc'))
    
    def on_epoch_end(self, batch, logs={}):
        dice = self.validate()
        self.dices.append([len(self.losses), dice])
        if dice > self.best_dice:
            self.best_dice = dice
            self.model.save(self.model_filename) # save best model to disk
            print('best model saved as {}'.format(self.model_filename))
        self.plot()   
    
    def validate(self):
        self.predictions = self.model.predict(self.val_imgs, batch_size=1)
        predicted_lbls = np.argmax(self.model.predict(self.val_imgs, batch_size=1), axis=3)
        x = self.val_lbls>0
        y = predicted_lbls>0
        return calculate_dice(x, y)
    
    def plot(self):
        clear_output()
        N = len(self.losses)
        plt.figure(figsize=(50, 10))
        plt.subplot(1, 5, 1)
        plt.plot(range(0, N), self.losses); plt.title('losses')
        plt.subplot(1, 5, 2)
        plt.plot(*np.array(self.dices).T); plt.title('dice')
        plt.subplot(1, 5, 3)
        plt.imshow(self.val_imgs[1]); plt.title('RGB image')
        plt.subplot(1, 5, 4)
        plt.imshow(np.argmax(self.predictions[1], axis=2)); plt.title('prediction')
        plt.subplot(1, 5, 5)
        plt.imshow(self.val_lbls[1]); plt.title('ground truth')
        plt.show()

Finally, let's define a function ``train_model`` that trains our model using training parameters and training/validation data:

In [ ]:
# function to train a model
def train_model(model, training_params):
    
    patch_size = training_params['patch_size']
    target_size = training_params['target_size']
    batch_size = training_params['batch_size']
    loss = training_params['loss']
    metrics = training_params['metrics']
    logger = training_params['logger']
    epochs = training_params['epochs']
    steps_per_epoch = training_params['steps_per_epoch']
    optimizer = training_params['optimizer']
    training_dataset = training_params['training_dataset']
    validation_dataset = training_params['validation_dataset']
        
    # batch generator 
    patch_generator = PatchExtractor(patch_size)
    batch_generator = BatchCreator(patch_generator, training_dataset, target_size=target_size)
    image_generator = batch_generator.get_image_generator(batch_size)

    # compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    # train the model
    model.fit_generator(generator=image_generator, 
                        steps_per_epoch=steps_per_epoch, 
                        epochs=epochs,
                        callbacks=[logger])

## Define datasets
Now we can use the tools that we have initialized to build the datasets that we will use in this assignment.

In [ ]:
# define directory for training and test data, as they were created after downloading data from SURFDrive
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# load images + manual annotations from the training set, images from the test set, and store them in lists.
# Note that 85 and 80 are hard-coded numbers here. You may have to change this if you change dataset in the future.
train_imgs = [load_img(os.path.join(train_dir, '{}.png'.format(f+1))) for f in range(85)]
train_lbls = [load_img(os.path.join(train_dir, '{}_anno.png'.format(f+1))) for f in range(85)]
test_imgs = [load_img(os.path.join(test_dir, '{}.png'.format(f))) for f in range(80)]

Images in the training set are currently in the same order as read from disk. To avoid any bias possibly introduced by this order, it is always good to shuffle the training dataset. Note that images and annotations must be shuffled in the same way!

In [ ]:
# shuffle order of training images and manual annotations
indexes = list(range(85))
random.shuffle(indexes)
train_imgs = list(np.asarray(train_imgs)[indexes])
train_lbls = list(np.asarray(train_lbls)[indexes])

### Split into training and validation set
Now we can define a training and a validation set by using the Dataset class that we have defined. In order to define a validation set, you have to specify a coefficient (from 0 to 1) to indicate the percentage of training images that you want to use for validation (we have seen that typical splits are 70/30, 80/20 etc.

In [ ]:
validation_percent = None # coefficient to define validation dataset (value between 0 and 1)

In [ ]:
n_validation_imgs = int(validation_percent * len(train_imgs))

# use the first images as validation
validation_dataset = DataSet(train_imgs[:n_validation_imgs], train_lbls[:n_validation_imgs])

# the rest as training
training_dataset = DataSet(train_imgs[n_validation_imgs:], train_lbls[n_validation_imgs:])

# test dataset
test_dataset = DataSet(test_imgs)

n_tra_imgs = training_dataset.get_lenght()
n_val_imgs = validation_dataset.get_lenght()
n_tes_imgs = test_dataset.get_lenght()

print('{} training images'.format(n_tra_imgs))
print('{} validation images'.format(n_val_imgs))
print('{} test images'.format(n_tes_imgs))

### Visualize datasets
Let's visualize images in the training, the validation and the test set.

In [ ]:
# training set
for i in range(n_tra_imgs):
    training_dataset.show_image(i)

In [ ]:
# validation set
for i in range(n_val_imgs):
    validation_dataset.show_image(i)

In [ ]:
# test set
for i in range(n_tes_imgs):
    test_dataset.show_image(i)

Note that manual annotations loaded from disk are in the range [0, 255], because they were stored as grayscale images, where white has an intensity of 255. For training purposes, we need to have labels in the range [0, 1]. Therefore, we have to apply the function ``binarize_dataset`` that we have defined before. After applying it, the range of values of manual annotations should be [0, 1].

In [ ]:
# set labels to 0 or 1 in training and validation datasets
training_dataset = binarize_dataset(training_dataset)
validation_dataset = binarize_dataset(validation_dataset)

Let's visualize the training and validation sets again to check that things are correct now.

In [ ]:
# visualize training set
for i in range(n_tra_imgs):
    training_dataset.show_image(i)

In [ ]:
# visualize validation set
for i in range(n_val_imgs):
    validation_dataset.show_image(i)

### Test batch generator
Before we delve into training U-Net, it is good to test whether the patch generator and the batch generator are working as expected (for example, check if the one-hot encoding is working as expected). Let's test them now.

In [ ]:
# define parameters for patch generator and batch creator
patch_size = (256, 256) # input size
target_size = (256, 256) # output size, might be the same as input size, but might be smaller, if valid convolutions are used
batch_size = 16 # number of patches in a mini-batch

# intialize patch generator and batch creator
patch_generator = PatchExtractor(patch_size)
batch_generator = BatchCreator(patch_generator, training_dataset, target_size=target_size)

# get one minibatch
x_data, y_data = batch_generator.create_image_batch(batch_size)

print(x_data.shape)
print(y_data.shape)

for i in range(batch_size):
    plt.subplot(1, 3, 1)
    plt.imshow(x_data[i]); plt.title('RGB image')
    plt.subplot(1, 3, 2)
    plt.imshow(pad_prediction(y_data[i, :, :, 0].squeeze(), patch_size)); plt.title('Label map class 0')
    plt.subplot(1, 3, 3)
    plt.imshow(pad_prediction(y_data[i, :, :, 1].squeeze(), patch_size)); plt.title('Label map class 1')
    plt.show()

Now we have all we need to start building our prediction model with training data.

# Train a baseline U-Net model (U-Net 1)
Here we explicitly define a baseline U-Net model. It has a depth of 4 (3 pooling layers), it uses 'same' convolutions, it has 16 filters in the first layer, it accepts inputs of 256x256 RGB images, so it is a very simplified version of the original U-Net. For convenience, we call it ``unet_1``, to differentiat it from other U-Net models that will be built in the rest of the assignment.

In [ ]:
def build_unet_1(printmodel=False):
    
    inputs = Input(shape=(256, 256, 3))

    # First conv pool
    c1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)

    # Second conv pool
    c2 = Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)

    # Third conv pool
    c3 = Conv2D(64, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(64, 3, activation='relu', padding='same')(c3)
    p3 = MaxPooling2D()(c3)

    # Fourth conv pool
    c4 = Conv2D(128, 3, activation='relu', padding='same')(p3)
    c4 = Conv2D(128, 3, activation='relu', padding='same')(c4)

    # First up-conv
    u2 = UpSampling2D()(c4)
    m2 = concatenate([c3, u2])
    cm2 = Conv2D(64, 3, activation='relu', padding='same')(m2)
    cm2 = Conv2D(64, 3, activation='relu', padding='same')(cm2)

    # Second up-conv
    u3 = UpSampling2D()(cm2)
    m3 = concatenate([c2, u3])
    cm3 = Conv2D(32, 3, activation='relu', padding='same')(m3)
    cm3 = Conv2D(32, 3, activation='relu', padding='same')(cm3)

    # Third up-conv
    u4 = UpSampling2D()(cm3)
    m4 = concatenate([c1, u4])
    cm4 = Conv2D(16, 3, activation='relu', padding='same')(m4)
    cm4 = Conv2D(16, 3, activation='relu', padding='same')(cm4)

    # Output
    predictions = Conv2D(2, 1, activation='softmax')(cm4)

    model = Model(inputs, predictions)
    
    if printmodel:
        print(model.summary())
    
    return model

Make an instance of the ``unet_1`` model.

In [ ]:
unet_1 = build_unet_1()

What are the performance of this U-Net with randomly initialized parameters? Let's find it out (at least, visually) by running the model on the validation set:

In [ ]:
# apply the model to the validation set
apply_model(unet_1, validation_dataset)

As you can see, the output of randomly initialized parameters is of course very bad.
In the next cell, we initialize all training parameters, define an image generator that will be able to return mini-batches during the training procedure, define a loss function, the mini-batch size, and other parameters needed to train U-Net. Then, we train the model using the ``train_model`` function defined above. Replace ``None`` with some values.

In [ ]:
# training parameters

model_name = 'unet_1'

training_params = {}
training_params['learning_rate'] = None
training_params['patch_size'] = (256, 256) # input size
training_params['target_size'] = (256, 256) # output size, might be the same as input size, but might be smaller, if valid convolutions are used
training_params['batch_size'] = None # number of patches in a mini-batch
training_params['steps_per_epoch'] = None # number of iterations per epoch
training_params['epochs'] = None # number of epochs

training_params['optimizer'] = SGD(lr=training_params['learning_rate'], momentum=0.9, nesterov=True)

training_params['loss'] = ['categorical_crossentropy']
training_params['metrics'] = ['accuracy']
training_params['training_dataset'] = training_dataset
training_params['validation_dataset'] = validation_dataset

# initialize a logger, to keep track of information during training
lost_border = ((training_params['patch_size'][0]-training_params['target_size'][0])//2, (training_params['patch_size'][1]-training_params['target_size'][1])//2)
training_params['logger'] = Logger(validation_dataset, lost_border, data_dir, model_name)

# train model
train_model(unet_1, training_params)

After training, we can load the best saved model and apply it to the validation set and check the performance:

In [ ]:
# load the best model (which gave best Dice score on validation set during training)
unet_1 = load_model(os.path.join(data_dir, model_name + '.h5'))

In [ ]:
# apply best model to the validation set first
apply_model(unet_1, validation_dataset)

Now we can use the function ``apply_model()`` also to process the test set and (optionally) make a submission file.

In [ ]:
# apply best model to test set and make a submission file
apply_model(unet_1, test_dataset, 'unet_1', True)

You can now download this zipfile with this link: [results.zip](results.zip).  
Next, upload your result to the challenge website (https://ismi-glas.grand-challenge.org/evaluation/results/).

You have realized that this baseline model does not perform well at all!
You can of course play around with ome hyperparameters (learning rate, mini-batch size, etc.) and try to improve performance. We tried a bit and with this architecture we could not get very good performance. The good news is that now you have an example and a full pipeline that you can use to build your own U-Net model and improve the performance! In the next cells, we will provide some guidance how to make U-Net work better, starting with adding batch normalization.

# Implement U-Net blocks and add Batch Normalization (U-Net 2)
You have probably realized that a U-Net model can be built by writing a simple program, because U-Net blocks have all the same structure, which can be coded as a function. Here we implement U-Net blocks, which will make the design of more U-Net architectures in this assignment much easier.

In [ ]:
# Create a function that builds a U-Net block, containing conv->(batchnorm->)conv->(batchnorm),
# where batchnorm is optional and can be selected via input parameter.
# The function returns the output of a convolutional (or batchnorm) layer "cl"
def unet_block(inputs, n_filters, batchnorm=False):
    
    # >> YOUR CODE HERE <<
    
    return cl

Use ``unet_block()`` to build a U-Net model with a script: 

In [ ]:
def build_unet_2(initial_filters=16, n_classes=2, batchnorm=False, printmodel=False):

    # build U-Net again using unet_block function
    inputs = Input(shape=(256, 256, 3))

    # CONTRACTION PART

    # First conv pool
    c1 = unet_block(inputs, initial_filters, batchnorm)
    p1 = MaxPooling2D()(c1)

    # Second conv pool
    c2 = unet_block(p1, 2*initial_filters, batchnorm)
    p2 = MaxPooling2D()(c2)

    # Third conv pool
    c3 = unet_block(p2, 4*initial_filters, batchnorm)
    p3 = MaxPooling2D()(c3)

    # Fourth conv
    c4 = unet_block(p3, 8*initial_filters, batchnorm)

    # EXPANSION PART

    # First up-conv
    u2 = UpSampling2D()(c4)
    m2 = concatenate([c3, u2])
    cm2 = unet_block(m2, 4*initial_filters, batchnorm)

    # Second up-conv
    u3 = UpSampling2D()(cm2)
    m3 = concatenate([c2, u3])
    cm3 = unet_block(m3, 2*initial_filters, batchnorm)

    # Third up-conv
    u4 = UpSampling2D()(cm3)
    m4 = concatenate([c1, u4])
    cm4 = unet_block(m4, initial_filters, batchnorm)

    # Output
    predictions = Conv2D(n_classes, 1, activation='softmax')(cm4)

    model = Model(inputs, predictions)
    
    if printmodel:
        print(model.summary())
    
    return model

We make a second model using the function we just made, when batch normalization is used:

In [ ]:
unet_2 = build_unet_2(batchnorm=True)

In [ ]:
# training parameters

model_name = 'unet_2'

training_params = {}
training_params['learning_rate'] = 0.0001
training_params['patch_size'] = (256, 256) # input size
training_params['target_size'] = (256, 256) # output size, might be the same as input size, but might be smaller, if valid convolutions are used
training_params['batch_size'] = 16 # number of patches in a mini-batch
training_params['steps_per_epoch'] = 10
training_params['epochs'] = 20
training_params['optimizer'] = SGD(lr=training_params['learning_rate'], momentum=0.9, nesterov=True)
training_params['loss'] = ['categorical_crossentropy']
training_params['metrics'] = ['accuracy']
training_params['training_dataset'] = training_dataset
training_params['validation_dataset'] = validation_dataset

# initialize a logger, to keep track of information during training
lost_border = ((training_params['patch_size'][0]-training_params['target_size'][0])//2, (training_params['patch_size'][1]-training_params['target_size'][1])//2)
training_params['logger'] = Logger(validation_dataset, lost_border, data_dir, model_name)

# train model
train_model(unet_2, training_params)

In [ ]:
# load the best model (which gave best Dice score on validation set during training)
unet_2 = load_model(os.path.join(data_dir, model_name + '.h5'))

In [ ]:
# apply model to test set and save results as results.zip
apply_model(unet_2, test_dataset, experiment_name='unet_2', make_submission_file=False)

Try to change the number of filters per layer by passing a different input parameter ``initial_filters`` (default = 16) to ``build_unet_2()``, and experiment the effect of using a wider U-Net model. Repeat the experiments done before (training and inference on the test set + submission to grand-challenge) by adding more cells below. **Do not modify previous cells, because this would make reading your assignment very difficult!**

In [ ]:
initial_filters = None
unet_2_wider = build_unet_2(initial_filters=initial_filters, batchnorm=True)

# Add Regularization (U-Net 3)
You have probably seen that batch normalization helps improve the performance. One reason could be because beatchnorm has some regularization effect. Maybe you have also noticed that we haven't used explicit regularization terms so far, as for example dropout layers, or L1/L2 losses. Now we want to do that and investigate how/whether performance improve.

For this purpose, make new functions ``unet_block_dropout`` and ``build_unt_3`` to add Dropout and L1/L2 regularization.

In [ ]:
def unet_block_dropout(inputs, n_filters, batchnorm=False, dropout_rate=0.5):
    
    cl = None
    
    # >>> YOUR CODE HERE <<<
    
    return cl

In [ ]:
def build_unet_3(initial_filters=16, n_classes=2, batchnorm=False, printmodel=False):
    
    # >>> YOUR CODE HERE <<<
    
    return model

# Add weight map
As seen in the lecture this week, U-Net uses weights in its loss function to take into account for (1) different frequency of labels and (2) distance of pixels from objects to segment. The first component is useful to compensate for skewed distributions of pixels in training samples (remember that in U-Net every pixel count as one sample in the loss function). The second component helps to enforce separation of objects in the segmentation map. In this assignment, we only focus on the first term, and we further simplify it by only computing the proportion of pixels in the entire training set, instead of for each training sample. For this purpose, the option ``class_weight`` of ``fit_generator()`` can be used (check out the Keras documentation here: https://keras.io/models/model/#fit_generator). Note that very small or very large weights can hamper the training procedure. This about the best way to define ``w_background`` and ``w_foreground``. For this purpose, you will have to modify the function ``train_model``, where ``fit_generator`` is used. When you modify the function, make sure that it is still compatible with the resto fo your code (all previous cells that do not use ``class_weigths``)!

In [ ]:
def get_class_weights(training_dataset):
    
    w_background = None
    w_foreground = None
    
    # >>> YOUR CODE HERE <<<
    
    return w_background, w_foreground

In the next cell, we assume you will be using ``unet_3`` as reference model, but feel free to use any other model you have developed so far, the main idea of this experiment is to check whether using class weights improve the performance of your model.

In [ ]:
# train model
train_model(unet_3, training_params)

# Use valid convolutions
So far, we have been using ``same`` convolutions in our U-Net model, which makes easy to control the concatenation of feature maps in the contraction and in the extraction paths. However, the original model didn't use same convolutions but instead used valid convolutions. This means that the netowkr architecture and the input size have to be considered at the same time, and set the proper parameters. Additionally, feature maps have to be cropped before concatenation.

The design of such a network requires considering several components and might be tricky. Therefore, we add this task as final and optional one, which will give you 10 extra points.

Concisdering that the (maximum) input size is 256x256, you should build a U-Net with:
* a depth of 4 (3 pooling layers)
* valid convolutions
* central cropping
* input size <= 256 (try to find the best combination)

In this final part of the assignment, you will have to use some of the functions that we have defined but not used yet, to pad and crop, and you should also modify the batch generator to return patches of different size (if different from 256); in that case, you can simply do central cropping in the batch generator.

In [ ]:
def unet_block_valid(inputs, n_filters, batchnorm=False, dropout_rate=0.5):
    
    cl = None
    
    # >>> YOUR CODE HERE <<<
    
    return cl

In [ ]:
def build_unet_valid():
    
    # >>> YOUR CODE HERE <<<
    
    return model